In [ ]:
import nest_asyncio

nest_asyncio.apply()

import os
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core import Settings
from llama_index.core import PromptTemplate

os.environ["OPENAI_API_KEY"] = ""

llm = OpenAI(model="gpt-4o-mini", temperature=0.0)
Settings.embed_model = OpenAIEmbedding(model_name=" text-embedding-ada-002")

documents = SimpleDirectoryReader("data").load_data()
splitter = SentenceSplitter(chunk_size=1024, chunk_overlap=20)
nodes = splitter.get_nodes_from_documents(documents)

df = pd.read_csv("questions/Lyft2021_queries.csv")
queries = df["Query"].tolist()

retriever = VectorStoreIndex(nodes)
query_engine = retriever.as_query_engine(
    similarity_top_k=2,
)

In [ ]:
from llama_index.core.evaluation import FaithfulnessEvaluator, RelevancyEvaluator

f_evaluator = FaithfulnessEvaluator(llm=llm)
r_evaluator = RelevancyEvaluator(llm=llm)
f_scores = []
r_scores = []

for query in queries:
    response = query_engine.query(query)

    f_score = f_evaluator.evaluate_response(response=response).score * 100
    r_score = r_evaluator.evaluate_response(query=query, response=response).score * 100
    f_scores.append(f_score)
    r_scores.append(r_score)

avg_f = sum(f_scores) / len(f_scores)
avg_r = sum(r_scores) / len(r_scores)

print("Average Faithfulness Score:", avg_f)
print("Average Relevancy Score:", avg_r)